<a href="https://colab.research.google.com/github/DrNOFX97/IronHack-AI-Engineering-Bootcamp/blob/main/lab_langchain_med_Gemma_2_9b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [1]:
import numpy as np
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use.

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text.

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available.

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [2]:
#news = pd.read_csv('labelled_newscatcher_dataset.csv', sep=';')
#MAX_NEWS = 1000
#DOCUMENT="title"
#TOPIC="topic"

news = pd.read_csv('bbc_news.csv')
MAX_NEWS = 1000
DOCUMENT="description"
TOPIC="title"

#news = pd.read_csv('articles.csv')
#MAX_NEWS = 100
#DOCUMENT="Article Body"
#TOPIC="Article Header"

#news = "PICK A DATASET" #Ideally pick one from the commented ones above

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [4]:
news["id"] = news.index
news.head()

,title,pubDate,guid,link,description,id
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...,0
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as...",1
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...,2
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...,3
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...,4


In [5]:
#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database.

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior.

In [6]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 M

In [7]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance.

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that.

In [8]:
chroma_client = chromadb.PersistentClient(path="/Users/f.nuno/IronHack/Lab/Week7")

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it.

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [9]:
collection_name = "news_collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)


It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***.
* In the **document** we store the big text, it's a different column in each Dataset.
* In **metadatas**, we can informa a list of topics.
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS.


In [10]:
from tqdm import tqdm

# Example data
subset_news_documents = subset_news[DOCUMENT].tolist()
subset_news_topics = subset_news[TOPIC].tolist()
max_news = MAX_NEWS

# Create a tqdm progress bar
progress_bar = tqdm(total=max_news, desc="Adding documents to collection")

# Iterate over the documents and add them to the collection
for idx in range(min(max_news, len(subset_news_documents))):
    document = subset_news_documents[idx]
    topic = subset_news_topics[idx]
    document_id = f"id{idx}"

    # Add document to collection
    collection.add(
        documents=[document],
        metadatas=[{TOPIC: topic}],
        ids=[document_id],
    )

    # Update progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

print("Documents added to collection.")

Adding documents to collection:   0%|          | 0/1000 [00:00<?, ?it/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 0.00/79.3M [00:00<?, ?iB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 1.00k/79.3M [00:00<4:46:27, 4.84kiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 17.0k/79.3M [00:00<47:01, 29.5kiB/s]  
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 34.0k/79.3M [00:00<29:20, 47.2kiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 51.0k/79.3M [00:01<26:01, 53.2kiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 67.0k/79.3M [00:01<21:04, 65.8kiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 75.0k/79.3M [00:01<24:22, 56.8kiB/s]
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:   0%|          | 85.0k/79.3M [00:01<25:42, 53.9kiB/s]
/roo

Documents added to collection.


In [11]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)


{'ids': [['id587', 'id862', 'id561', 'id295', 'id653', 'id312', 'id622', 'id925', 'id21', 'id280']], 'distances': [[1.6517921686172485, 1.6726526021957397, 1.7190912961959839, 1.7223870754241943, 1.7251734733581543, 1.7306060791015625, 1.7306060791015625, 1.7306060791015625, 1.7306283712387085, 1.748883843421936]], 'metadatas': [[{'title': 'How a jetpack design helped create a flying motorbike'}, {'title': "Simon Coveney's visit to Belfast interrupted by security alert"}, {'title': 'Child, 13, drove pickup in deadly Texas crash that killed nine'}, {'title': 'Covid infections rising again across UK - ONS'}, {'title': 'UK holds plane with possible Russian ties'}, {'title': 'Week in pictures: 5-11 March 2022'}, {'title': 'Week in pictures: 12 - 18 March 2022'}, {'title': 'Week in pictures: 19 - 25 March 2022'}, {'title': 'War in Ukraine: Russian helicopter shot down by missile'}, {'title': 'Formula 1 testing: Red Bull unhappy with Mercedes upgrade in Bahrain'}]], 'embeddings': None, 'docu

## Vector MAP

In [12]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [13]:

getado = collection.get(ids="id141",
                       include=["documents", "embeddings"])


In [14]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

[[0.08360528200864792,
  0.009881743229925632,
  -0.06335217505693436,
  0.006835514679551125,
  0.09836876392364502,
  0.06623635441064835,
  0.08818849176168442,
  0.004320137668401003,
  0.046258460730314255,
  -0.025371838361024857,
  0.030316133052110672,
  0.02904367633163929,
  0.037352293729782104,
  0.017455749213695526,
  -0.03087661787867546,
  -0.0447610504925251,
  -0.007606553845107555,
  -0.05808791145682335,
  -0.04539887234568596,
  -0.06970192492008209,
  -0.003506844164803624,
  0.04581326246261597,
  0.015820123255252838,
  0.03446853533387184,
  -0.06203140318393707,
  0.09383269399404526,
  0.0595623217523098,
  -0.03075232170522213,
  -0.08414644747972488,
  -0.025589847937226295,
  -0.005449125077575445,
  -0.016466127708554268,
  -0.10067088156938553,
  0.05222402885556221,
  -0.006522803101688623,
  0.06448018550872803,
  0.062198929488658905,
  -0.004692091606557369,
  -0.03388358652591705,
  -0.0005861778045073152,
  -0.09118968993425369,
  -0.09242996573448

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents.

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search.


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models.

We are importing:
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification.

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2.

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [15]:
!pip install --upgrade transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

The next step is to initialize the pipeline using the objects created above.

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [16]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    #device_map="auto",
    device=0,
    )

## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user.

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**.

We only need to join the two parts together to create the prompt that we are going to send to the model.

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part.

In [17]:
question = "Can I buy a Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

"Relevant context: #The developers of a powerful mini aircraft hope it will be used by the armed forces. #A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out. #A 13-year-old was driving a truck that collided with a van carrying student athletes, officials say. #A form of Omicron called BA.2 is now the most common variant in most of the UK, figures suggest. #The jet is being held at London Biggin Hill airport while it is investigated if it falls under UK sanctions. #A selection of powerful images from all over the globe, taken in the past seven days. #A selection of powerful images from all over the globe, taken in the past seven days. #A selection of powerful images from all over the globe, taken in the past seven days. #The video, shared by Ukrainian armed forces, shows the helicopter being hit by a missile. #Mercedes are at the centre of a row over the legality of their car after introducing a radical new design at the second p

Now all that remains is to send the prompt to the model and wait for its response!


In [18]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Relevant context: #The developers of a powerful mini aircraft hope it will be used by the armed forces. #A van was reportedly hijacked and driven to the venue, and a controlled explosion has since been carried out. #A 13-year-old was driving a truck that collided with a van carrying student athletes, officials say. #A form of Omicron called BA.2 is now the most common variant in most of the UK, figures suggest. #The jet is being held at London Biggin Hill airport while it is investigated if it falls under UK sanctions. #A selection of powerful images from all over the globe, taken in the past seven days. #A selection of powerful images from all over the globe, taken in the past seven days. #A selection of powerful images from all over the globe, taken in the past seven days. #The video, shared by Ukrainian armed forces, shows the helicopter being hit by a missile. #Mercedes are at the centre of a row over the legality of their car after introducing a radical new design at the second pr